# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv file city_data_output.csv
output_csv_file = "../WeatherPy/city_data_output.csv"

# Create dataframe
weather_df = pd.read_csv(output_csv_file)

# View dataframe
weather_df = weather_df.drop(columns=["Unnamed: 0"])
weather_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,mayumba,GA,79.81,77.0,98.0,5.30,-3.4320,10.6554
1,punta arenas,CL,41.11,56.0,0.0,17.27,-53.1500,-70.9167
2,rikitea,PF,77.02,74.0,72.0,23.62,-23.1203,-134.9692
3,qaanaaq,GL,19.09,70.0,16.0,5.21,77.4840,-69.3632
4,lebu,CL,52.43,78.0,98.0,12.66,-37.6167,-73.6500
...,...,...,...,...,...,...,...,...
600,bichura,RU,40.08,40.0,90.0,1.10,50.5881,107.6022
601,nantucket,US,54.95,83.0,100.0,12.66,41.2835,-70.0995
602,samarinda,ID,74.48,97.0,100.0,0.98,-0.5000,117.1500
603,manggar,ID,82.08,80.0,99.0,2.59,-2.8833,108.2667


In [3]:
# Drop any NA values from dataframe
weather_df=weather_df.dropna()
weather_df


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,mayumba,GA,79.81,77.0,98.0,5.30,-3.4320,10.6554
1,punta arenas,CL,41.11,56.0,0.0,17.27,-53.1500,-70.9167
2,rikitea,PF,77.02,74.0,72.0,23.62,-23.1203,-134.9692
3,qaanaaq,GL,19.09,70.0,16.0,5.21,77.4840,-69.3632
4,lebu,CL,52.43,78.0,98.0,12.66,-37.6167,-73.6500
...,...,...,...,...,...,...,...,...
600,bichura,RU,40.08,40.0,90.0,1.10,50.5881,107.6022
601,nantucket,US,54.95,83.0,100.0,12.66,41.2835,-70.0995
602,samarinda,ID,74.48,97.0,100.0,0.98,-0.5000,117.1500
603,manggar,ID,82.08,80.0,99.0,2.59,-2.8833,108.2667


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

In [5]:
print(min(humidity))

5.0


In [7]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Find cities with Temperature greater than 70 and less than or equal to 80. Windspeed less than 10 mph
# and 0 cloudiness.
target_city_df = weather_df.loc[(weather_df["Temperature"] > 70) & (weather_df["Temperature"] < 80) & 
                                (weather_df["Cloudiness"] == 0) & weather_df["Windspeed"] <10, :]

temperature = (target_city_df["Temperature"] <= 80) & (target_city_df["Temperature"] > 70)
windspeed = target_city_df["Windspeed"] < 10
cloudiness = target_city_df["Cloudiness"] == 0

target_city_df=target_city_df[temperature & windspeed & cloudiness]

# Display updated dataframe
target_city_df



,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
66,cape town,ZA,71.22,50.0,0.0,9.22,-33.9258,18.4232
78,cabo san lucas,MX,76.51,45.0,0.0,8.59,22.8909,-109.9124
193,makakilo city,US,70.09,78.0,0.0,5.75,21.3469,-158.0858
312,nanakuli,US,71.22,75.0,0.0,5.75,21.3906,-158.1547
428,nsoko,SZ,78.06,67.0,0.0,4.45,-27.0333,31.9500
435,marsa matruh,EG,73.60,60.0,0.0,6.91,31.3525,27.2453
443,corinto,GR,70.90,31.0,0.0,3.56,37.9407,22.9573
450,kruisfontein,ZA,71.89,45.0,0.0,2.26,-34.0033,24.7314
473,todos santos,MX,70.74,53.0,0.0,5.59,23.4500,-110.2167
516,la libertad,EC,78.46,73.0,0.0,5.75,-2.2333,-80.9000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create a new dataframe to find hotels in the desired cities
hotel_df = target_city_df[["City","Country","Latitude","Longitude"]].copy()

hotel_df


,City,Country,Latitude,Longitude
66,cape town,ZA,-33.9258,18.4232
78,cabo san lucas,MX,22.8909,-109.9124
193,makakilo city,US,21.3469,-158.0858
312,nanakuli,US,21.3906,-158.1547
428,nsoko,SZ,-27.0333,31.9500
435,marsa matruh,EG,31.3525,27.2453
443,corinto,GR,37.9407,22.9573
450,kruisfontein,ZA,-34.0033,24.7314
473,todos santos,MX,23.4500,-110.2167
516,la libertad,EC,-2.2333,-80.9000


In [11]:
# Add a column titled "Hotel Name" to dataframe
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
66,cape town,ZA,-33.9258,18.4232,NaN
78,cabo san lucas,MX,22.8909,-109.9124,NaN
193,makakilo city,US,21.3469,-158.0858,NaN
312,nanakuli,US,21.3906,-158.1547,NaN
428,nsoko,SZ,-27.0333,31.9500,NaN
435,marsa matruh,EG,31.3525,27.2453,NaN
443,corinto,GR,37.9407,22.9573,NaN
450,kruisfontein,ZA,-34.0033,24.7314,NaN
473,todos santos,MX,23.4500,-110.2167,NaN
516,la libertad,EC,-2.2333,-80.9000,NaN


In [12]:
# Set parameters
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# Set up dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# Define URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [13]:
# Set up loop to pull hotel info for each city

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    
    hotel_data = requests.get(base_url, params).json()
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"The nearest hotel in '{row['City']}' is {hotel_name}.")
        print("---------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'.")    
        print("-----------------------------------------------------")

The nearest hotel in 'cape town' is The Table Bay hotel.
---------------------------------------------------
The nearest hotel in 'cabo san lucas' is Medano Hotel and Suites.
---------------------------------------------------
The nearest hotel in 'makakilo city' is Hampton Inn & Suites Oahu/Kapolei.
---------------------------------------------------
Could not find information for city 'nanakuli'.
-----------------------------------------------------
Could not find information for city 'nsoko'.
-----------------------------------------------------
The nearest hotel in 'marsa matruh' is Almanara Hotel Marsa Matrouh.
---------------------------------------------------
The nearest hotel in 'corinto' is Club Hotel Casino Loutraki.
---------------------------------------------------
The nearest hotel in 'kruisfontein' is Humansdorp Boutique Hotel.
---------------------------------------------------
The nearest hotel in 'todos santos' is Hotel San Cristóbal.
--------------------------------

In [14]:
# Remove any null values
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# Show dataframe
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,cape town,ZA,-33.9258,18.4232,The Table Bay hotel
1,cabo san lucas,MX,22.8909,-109.9124,Medano Hotel and Suites
2,makakilo city,US,21.3469,-158.0858,Hampton Inn & Suites Oahu/Kapolei
3,marsa matruh,EG,31.3525,27.2453,Almanara Hotel Marsa Matrouh
4,corinto,GR,37.9407,22.9573,Club Hotel Casino Loutraki
5,kruisfontein,ZA,-34.0033,24.7314,Humansdorp Boutique Hotel
6,todos santos,MX,23.4500,-110.2167,Hotel San Cristóbal
7,la libertad,EC,-2.2333,-80.9000,Playa Canela Hotel Boutique


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Display heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))